In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
folder = '/kaggle/input/happy-mammals-with-128x128-image-size/train_images_128'
images = []
# for dirname, _, filenames in os.walk(folder):
#     for filename in filenames:
# #         print(os.path.join(dirname, filename))

# print("load train data sucsess")
# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Visualize data

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import os
import cv2
path = '/kaggle/input/happy-whale-and-dolphin/'
dir = os.listdir(path)
print(dir)
train_data = pd.read_csv(path+'train.csv')
print('Number in train.csv:', len(train_data))

In [ ]:
#print sample of train.csv
train_data.head()

In [ ]:
train_data.loc[train_data['image'] == '80b5373b87942b.jpg']

In [ ]:
print('Number train images:', len(os.listdir(path+'train_images/')))
print('Number test images:', len(os.listdir(path+'test_images/')))

In [ ]:
count_species=train_data['species'].value_counts()
print(count_species)
print(f"\nTotal number of species: {len(count_species)}")

In [ ]:
plt.figure(figsize=(20, 15))
plt.rcParams["font.size"] = 18
plt.barh(train_data["species"].value_counts().sort_values(ascending=True).index,
         train_data["species"].value_counts().sort_values(ascending=True),
         tick_label = train_data["species"].value_counts().sort_values(ascending=True).index)
plt.show()

In [ ]:
train_data['label'] = train_data.species.map(lambda x: 'whale' if 'whale' in x else 'dolphin')
data = train_data['label'].value_counts().reset_index()
print(data)

In [ ]:
# print(data.iloc[0:, 0])
plt.figure(figsize=(15, 10))
plt.rcParams["font.size"] = 12
plt.title("Dolphin and whale")
plt.bar(data.iloc[0:, 0], data.iloc[0:, 1], color=['r','b'])
plt.show()

In [ ]:
PATH_ = "../input/happy-whale-and-dolphin/train_images/"
image = plt.imread(PATH_+train_data.iloc[0, 0])
plt.figure(figsize=(15, 6))
plt.imshow(image)

print(train_data.iloc[0])

In [ ]:
image = plt.imread(PATH_+train_data.iloc[150, 0])
plt.figure(figsize=(15, 6))
plt.imshow(image)

print(train_data.iloc[0])

In [ ]:
PATH_ = "../input/happy-whale-and-dolphin/train_images/"
image = plt.imread(PATH_+train_data.iloc[25492, 0])
plt.figure(figsize=(15, 6))
plt.imshow(image)

print(train_data.iloc[0])

# Clean data and visualize label

In [ ]:
# Clean data
train_df = pd.read_csv("../input/happy-whale-and-dolphin/train.csv")
train_df.shape

In [ ]:
train_df.head()

In [ ]:
train_df.isnull()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df['individual_id'].value_counts()

In [ ]:
# plt.figure(figsize=(20, 15))
# plt.rcParams["font.size"] = 18
# plt.barh(train_df["individual_id"].value_counts().sort_values(ascending=True).index,
#          train_df["individual_id"].value_counts().sort_values(ascending=True))
# plt.show()

# Prepare data for model

In [ ]:
#Function
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
def Loading_Images(data, m, dataset):
    print("Loading images")
    X_train = np.zeros((m, 32, 32, 3))
    count = 0
    for fig in tqdm(data['image']):
        img = image.load_img("../input/happy-whale-and-dolphin/"+dataset+"/"+fig, target_size=(32, 32, 3))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        X_train[count] = x
        count += 1
    return X_train

In [ ]:
X = Loading_Images(train_df, train_df.shape[0], "train_images")

In [ ]:
#Nomarlize data
X /= 255

In [ ]:
X.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    y = onehot_encoded
    return y, label_encoder

In [ ]:
y, label_encoder = prepare_labels(train_df['individual_id'])

In [ ]:
y.shape

# Train Model

In [ ]:
import numpy as np
import pandas as pd

import os
from glob import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras import layers
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

model = Sequential()

model.add(Conv2D(32, (6, 6), strides = (1, 1), input_shape = (32, 32, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), strides = (1,1)))
model.add(Activation('relu'))
model.add(AveragePooling2D((3, 3)))

model.add(Flatten())
model.add(Dense(512, activation="relu"))

model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X, y, epochs=150, batch_size=128, verbose=1)

In [ ]:
model.save('model.h5')

In [ ]:
# แสดงค่าความแม่นยำในรูปแบบกราฟ

plt.figure(figsize=(15,5))
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [ ]:
# แสดงค่า Model loss

plt.figure(figsize=(15,5))
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.show()

# เตรียมใช้โมเดลกับ test_images

In [ ]:
test = os.listdir("../input/happy-whale-and-dolphin/test_images")
print(len(test))

In [ ]:
# เตรียม Variable เพื่อใช้โมเดล predict ใน format ที่การแข่งขันต้องการ
col = ['image']
test_df = pd.DataFrame(test, columns=col)
test_df['predictions'] = ''

In [ ]:
batch_size=5000
batch_start = 0
batch_end = batch_size
L = len(test_df)

while batch_start < L:
    limit = min(batch_end, L)
    test_df_batch = test_df.iloc[batch_start:limit]
    print(type(test_df_batch))
    X = Loading_Images(test_df_batch, test_df_batch.shape[0], "test_images")
    X /= 255
    predictions = model.predict(np.array(X), verbose=1)
    for i, pred in enumerate(predictions):
        p=pred.argsort()[-5:][::-1]
        idx=-1
        s=''
        s1=''
        s2=''
        for x in p:
            idx=idx+1
            if pred[x]>0.6:
                s1 = s1 + ' ' +  label_encoder.inverse_transform(p)[idx]
            else:
                s2 = s2 + ' ' + label_encoder.inverse_transform(p)[idx]
        s= s1 + ' new_individual' + s2
        s = s.strip(' ')
        test_df.loc[ batch_start + i, 'predictions'] = s
    batch_start += batch_size   
    batch_end += batch_size

In [ ]:
# สร้างไฟล์ CSV เพื่อเตรียม submit

test_df.to_csv('submission.csv',index=False)
test_df.head()

# Evaluation

In [ ]:
evaluate_df = pd.read_csv("../input/happy-whale-and-dolphin/train.csv", nrows=10000)
eva = Loading_Images(evaluate_df, evaluate_df.shape[0], "train_images")

In [ ]:
evaluate_df = pd.read_csv("../input/happy-whale-and-dolphin/train.csv")
eva_y, label_encoder = prepare_labels(evaluate_df['individual_id'])
print(len(eva_y))

In [ ]:
import tensorflow as tf
from tensorflow import keras
reconstructed_model = keras.models.load_model("../input/model-for-evaluation-happywhale/model.h5")

In [ ]:
eva/=255
x = np.array(eva)
predict = reconstructed_model.predict(x, verbose=1)
ans = label_encoder.inverse_transform([0])

In [ ]:
p=predict[0].argsort()[-1:]      #sort index and pick the most values but as it index ofcouse
print(len(predict[0]))
print(len(eva_y[0]))
ans = label_encoder.inverse_transform(p)
print("Predict = ",ans)

print(p)
y=eva_y[0].argsort()[-1:]
print(y)
y_ans = label_encoder.inverse_transform(y)
print("actual = ",y_ans)

count=0;
for i, pred in enumerate(predict):
    p=pred.argsort()[-1:] 
    y=eva_y[i].argsort()[-1:] 
    if(p[0] == y[0]):
        count+=1
        print("equal_index = ",i)
print("total = ",count)
print("total test = ", len(predict))
print("Acculancy percentage = ",count/len(predict))

In [ ]:
#Check if equal_index true
p=predict[1].argsort()[-1:]      #sort index and pick the most values but as it index ofcouse
print(len(predict[0]))
print(len(eva_y[0]))
ans = label_encoder.inverse_transform(p)
print("Predict = ",ans)

print(p)
y=eva_y[1].argsort()[-1:]
print(y)
y_ans = label_encoder.inverse_transform(y)
print("actual = ",y_ans)

In [ ]:
#validate y label
y_i=eva_y[0].argsort()[-1:]
print("sum of y at index 0 = ",eva_y[0].sum())
print("max value(as an index) at index 0 = ",y_i)
print("y 0 at max value(as an index) = ", eva_y[0][y_i])

In [ ]:
#Evaluation with 5 inputs
count_s=0;
for i, pred in enumerate(predict):
    p=pred.argsort()[-5:][::-1] 
    y=eva_y[i].argsort()[-1:]
    for j in range(5):
        if(p[j] == y[0]):
            count_s+=1
            print("equal_index = "+str(i)+" and number of predict that correct = "+str(j))
            continue
print("total in 5 ans = ",count_s)
print("total test = ", len(predict))
print("Accuracy = ", count_s/len(predict))

In [ ]:
#Evaluate with MAP@5
count_m5=0;
sumofPrecis=[];
for i, pred in enumerate(predict):
    p=pred.argsort()[-5:][::-1] 
    y=eva_y[i].argsort()[-1:]
    for j in range(5):
        if(p[j] == y[0]):
            count_m5+=1
            sumofPrecis.append(pred[p[j]])
            print("equal_index = "+str(i)+" and number of predict that correct = "+str(j)+" precission = ", pred[p[j]])
            continue
print("total in 5 ans = ",count_m5)
print("total test = ", len(predict))
print("Accuracy = ", count_m5/len(predict))
print("Sum of best 5 correct precission(MAP@5) = ", sum(sumofPrecis))
print("Accuracy(MAP@5) = ", sum(sumofPrecis)/len(predict))

# TEST

In [ ]:
# เตรียม Variable เพื่อใช้โมเดล predict ใน format ที่การแข่งขันต้องการ
# col = ['image']
# test_dff = pd.DataFrame(test, columns=col)
# test_dff['predictions'] = ''
# test_dff_batch = test_dff.iloc[0:1]
# test_dff_batch

# X = Loading_Images(test_dff_batch, test_dff_batch.shape[0], "test_images")
# X /= 255

# predictions = model.predict(np.array(X), verbose=1)
# np.set_printoptions(suppress=True)
# print('last value = ',predictions[0][len(predictions[0])-1])
# print('max = ',np.max(predictions[0]))
# print('min = ',np.min(predictions[0]))

# for i, pred in enumerate(predictions):
#         p=pred.argsort()[-5:][::-1] #argsort basicly return index of sorting increasing array
#         print(p)
#         print(pred.argsort()[-5:])
#         print(pred.argsort())
#         print('test min? = ', pred[9483])

In [ ]:
# ss = np.array([1.48,1.41,0.0,0.1])
# print(ss.argsort())